In [1]:
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup, Comment
import requests

In [2]:
# Firstly we need to collect all links that we intend to scrape from
# the data needed is in the format of:
# https://www.pro-football-reference.com/years/2018/week_21.htm
# We need to set up a dictionary containing all weeks and years: seasonDict

# First setup years and weeks we are interested in. Note that the
# postseason starts in week 18 and the superbowl is week 21
# We will collect dating back to 2004
yearNum = [year for year in range(2007,2019)]
weekNum = [week for week in range(1,17)]

seasonDict = {i : [j for j in weekNum] for i in yearNum}
print(seasonDict)

{2016: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], 2017: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], 2018: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], 2007: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], 2008: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], 2009: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], 2010: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], 2011: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], 2012: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], 2013: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], 2014: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], 2015: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]}


In [3]:
# Final cleaned script

csvFile = open("./csvFiles/dvoaData2.csv", 'w+')
writer = csv.writer(csvFile)
writer.writerow(('season','week','team','totalDvoa','offDvoa', 'offRank', 'defDvoa', 'defRank', 'stDvoa','stRank'))



def getRows(bsO,season,week):
    rows = bsO.find('table', {'border':'2'}).findAll('tr')
    season = season
    week = week
    for row in rows[1:17]:
        #collect each column of data needed, THEN write each column into a row in the .csv file
        attVals = row.findAll('td')
        team = attVals[1].get_text()
        totalDvoa = attVals[2].get_text()
        offDvoa = attVals[-6].get_text()
        offRank = attVals[-5].get_text()
        defDvoa = attVals[-4].get_text()
        defRank = attVals[-3].get_text()
        stDvoa = attVals[-2].get_text()
        stRank = attVals[-1].get_text()
        #now write the row of data
        writer.writerow((season,week,team,totalDvoa,offDvoa,offRank,defDvoa,defRank,stDvoa,stRank))
        
    for row in rows[18:]:
        # Repeat for second half of the table
        attVals = row.findAll('td')
        team = attVals[1].get_text()
        totalDvoa = attVals[2].get_text()
        offDvoa = attVals[-6].get_text()
        offRank = attVals[-5].get_text()
        defDvoa = attVals[-4].get_text()
        defRank = attVals[-3].get_text()
        stDvoa = attVals[-2].get_text()
        stRank = attVals[-1].get_text()
        #now write the row of data
        writer.writerow((season,week,team,totalDvoa,offDvoa,offRank,defDvoa,defRank,stDvoa,stRank))
        
    
#cycle thru all years/weeks    
for season in seasonDict.keys():
    for week in seasonDict[season]:
        # There seems to be an issue scraping the first and last weeks of the 2018 season, so I will
        # scrape them manually and merge that data to the data from this scraper when it is finished running
        if season == 2018 and (week == 1 or week == 16):
            pass
        else:
            html = urlopen('http://www.footballoutsiders.com/dvoa-ratings/' + str(season) + '/week-' + str(week) + '-dvoa-ratings')
            bsObj = BeautifulSoup(html, 'html5lib')
            getRows(bsObj,season,week)